In [47]:
def all_bits(num_bits: int):
    return (1 << num_bits) - 1

In [48]:
def bit_not(n):
    return ~n & all_bits(32)

In [49]:
def ch(x: int, y: int, z: int):
    return ( x & y ) ^ ( ~x & z )

In [50]:
def maj(x: int, y: int, z: int):
    return ( x & y ) ^ ( x & z ) ^ ( y & z )

In [51]:
def rot_right(x: int, b: int, bits=32):
    return (x >> b) | (x << bits - b)

In [52]:
def cap_sigma_0(x: int):
    return rot_right(x,2) ^ rot_right(x,13) ^ rot_right(x,22)

In [53]:
def cap_sigma_1(x: int):
    return rot_right(x,6) ^ rot_right(x,11) ^ rot_right(x,25)

In [54]:
def sigma_0(x: int):
    return rot_right(x,7) ^ rot_right(x, 18) ^ ( x >> 3 )

In [55]:
def sigma_1(x: int):
    return rot_right(x,17) ^ rot_right(x,19) ^( x >> 10 )

In [56]:
# Konstanten für die Rundenfunktion
k = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]
len(k)

64

In [57]:
message = 'Hello World!'

In [58]:
message = bytearray(message, 'ascii')
length = len(message) * 8

In [59]:
message.append(0b10000000) # Byte mit führender 1 anhängen

In [60]:
# Nullen anhängen/auffüllen bis zum Block für die Länge
while ( len(message) * 8 + 64 ) % 512 != 0:
    message.append(0b00000000) # Null anhängen

In [61]:
# Länge berechnen
length_block_value = length.to_bytes(8, 'big') # 8 Bytes = 64 Bits

# Länge anhängen
message += length_block_value

In [62]:
len(message) * 8

512

In [63]:
for (i,value) in enumerate(''.join('{:02x}'.format(x) for x in message)):
    e = "" if (i+1) % 8 else " "
    e = e if (i+1) % 64 else "\n"
    print(value, end = e)

48656c6c 6f20576f 726c6421 80000000 00000000 00000000 00000000 00000000
00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000060


In [64]:
assert (len(message) * 8) % 512 == 0, "Padding error!"

In [71]:
blocks = []
for i in range(0, len(message), 64): # 64 Bytes sind 512 Bits
    blocks.append(message[i:i+64]) # 64 Bytes / 512 Bits als einzelnen BLock aufnehmen
    
print(blocks)
print(type(blocks[0]))

[bytearray(b'Hello World!\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00`')]
<class 'bytearray'>


In [66]:
# Hash-Werte mit IV initialisieren
h0=0x6a09e667
h1=0xbb67ae85
h2=0x3c6ef372
h3=0xa54ff53a
h4=0x510e527f
h5=0x9b05688c
h6=0x1f83d9ab
h7=0x5be0cd19

In [67]:
# Jeden Block separate bearbeiten
for block in blocks:

    # Zwischenspeicher für erweiterten Block initialisieren
    block_exp = []

    # Jeden Block in 64 Teile mit 32 Bit unterteilen
    for t in range(0,64):

        # Die ersten 16 Teile einfach nur anhängen
        if t < 16:
            block_exp.append(bytes(block[t*4:(t*4)+4]))

        # Die restlichen 48 anhand der gegebenen Formel erzeugen
        else:
            # Die einzelnen Werte ermitteln
            wim02 = int.from_bytes(block_exp[t-2] ,'big')
            wim07 = int.from_bytes(block_exp[t-7] ,'big')
            wim15 = int.from_bytes(block_exp[t-15],'big')
            wim16 = int.from_bytes(block_exp[t-16],'big')

            # Den Wert W von i anhand der vorgegebenen Formel berechnen
            wi = sigma_1(wim02) + wim07 + sigma_0(wim15) + wim16

            # Den Wert auf 32 Bit bringen und in Big Endian konvertieren
            exp = int(wi % 2**32).to_bytes(4, 'big')

            # Block anhängen
            block_exp.append(exp)

    assert len(block_exp) == 64

    # Das Arbeitsregister mit den ersten Werten h0-h7 initialisieren
    a = h0
    b = h1
    c = h2
    d = h3
    e = h4
    f = h5
    g = h6
    h = h7

    # Register 64-mal aktualisieren
    for t in range(64):

        kj = k[t]
        wj = int.from_bytes(block_exp[t],'big')

        t1 = ( h + cap_sigma_1(e) + ch(e,f,g) + kj + wj ) % 2**32
        t2 = ( cap_sigma_0(a) + maj(a,b,c) ) % 2**32

        h = g
        g = f
        f = e
        e = ( d + t1 ) % 2**32
        d = c
        c = b
        b = a
        a = ( t1 + t2 ) % 2**32

    # Den i-ten zwischenzeitlichen Hashwert H(i) ermitteln
    h0 = ( a + h0 ) % 2**32
    h1 = ( b + h1 ) % 2**32
    h2 = ( c + h2 ) % 2**32
    h3 = ( d + h3 ) % 2**32
    h4 = ( e + h4 ) % 2**32
    h5 = ( f + h5 ) % 2**32
    h6 = ( g + h6 ) % 2**32
    h7 = ( h + h7 ) % 2**32

hash_value = (
    h0.to_bytes(4, 'big') + \
    h1.to_bytes(4, 'big') + \
    h2.to_bytes(4, 'big') + \
    h3.to_bytes(4, 'big') + \
    h4.to_bytes(4, 'big') + \
    h5.to_bytes(4, 'big') + \
    h6.to_bytes(4, 'big') + \
    h7.to_bytes(4, 'big') )
    
hash_value.hex()

'7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069'

In [68]:
for (i,value) in enumerate(hash_value.hex()):
    e = "" if (i+1) % 8 else " "
    e = e if (i+1) % 64 else "\n"
    print(value, end = e)

7f83b165 7ff1fc53 b92dc181 48a1d65d fc2d4b1f a3d67728 4addd200 126d9069
